In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from scipy.optimize import curve_fit

In [37]:
def write_potential_to_lowdin_format(potentialName,species,angularMomentum,exponents,coefficients,centers):
    
    outFile = open(potentialName, "w")
    
    outFile.write("O-"+species+"\n")
    outFile.write("#\n")
    outFile.write("%i\n" % (len(exponents)))
    
    for i in range(0,len(exponents)) :
        outFile.write( str(i+1) + " "+ str(int(angularMomentum[i]))+"\n" )
        outFile.write( "%.8f %.8e\n" % (exponents[i], coefficients[i]) )
        
        aux = str(centers[i][0])+" "+str(centers[i][1])+" "+str(centers[i][2])
    
        outFile.write( aux + "\n")

    outFile.close()

    
def join_potentials(first_potential,second_potential,resulting_potential,species):
    
    #Read first potential and obtain its number of gaussians
    first_pot = open(first_potential, "r")
    lines_fp = first_pot.readlines()
    num_gauss_fp = int(lines_fp[2])
    
    #Read second potential and obtain its number of gaussians
    second_pot = open(second_potential, "r")
    lines_sp = second_pot.readlines()
    num_gauss_sp = int(lines_sp[2])
    
    #Declare total number of gaussians 
    num_gauss_result = num_gauss_fp + num_gauss_sp
    
    #Declare exponents, coefficients, centers and angular momentum list
    exponents = []
    coefficients = []
    centers = []
    angular_momentum = []
    
    #Extract exponents and coefficients
    #########################################################
    for i in range(4,len(lines_fp),3):
        exponents.append(float(lines_fp[i].split()[0]))
        coefficients.append(float(lines_fp[i].split()[1]))
    
    for i in range(4,len(lines_sp),3):
        exponents.append(float(lines_sp[i].split()[0]))
        coefficients.append(float(lines_sp[i].split()[1]))
    
    #########################################################
    
    
    #Extract centers
    #########################################################
    for i in range(5, len(lines_fp),3):
        centers.append([lines_fp[i].split()[0],lines_fp[i].split()[1],lines_fp[i].split()[2]])
        
    for i in range(5,len(lines_sp),3):
        centers.append([float(lines_sp[i].split()[0]),float(lines_sp[i].split()[1]),float(lines_sp[i].split()[2])])
    #########################################################
    
    
    #Extract angular momentum
    ##########################################################
    for i in range(3, len(lines_fp),3):
        angular_momentum.append(int(lines_fp[i].split()[1]))
    
    for i in range(3, len(lines_sp),3):
        angular_momentum.append(int(lines_sp[i].split()[1]))
    
    ##########################################################
    
    
    write_potential_to_lowdin_format(resulting_potential,species,angular_momentum,exponents,coefficients,centers)
    
    
    

In [40]:
datapath="dataHminus2.dat"


def gaussians2(r,a_0,a_1,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9,b_0,b_1,b_2,b_3,b_4,b_5,b_6,b_7,b_8,b_9):
	return a_0*np.exp(-b_0*(r)**2)+a_1*np.exp(-b_1*(r)**2)+a_2*np.exp(-b_2*(r)**2)+a_3*np.exp(-b_3*(r)**2)+a_4*np.exp(-b_4*(r)**2)+a_5*np.exp(-b_5*(r)**2)+a_6*np.exp(-b_6*(r)**2)+a_7*np.exp(-b_7*(r)**2)+a_8*np.exp(-b_8*(r)**2)+a_9*np.exp(-b_9*(r)**2)

def gaussians3(r,a_0,a_1,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9,b_0,b_1,b_2,b_3,b_4,b_5,b_6,b_7,b_8,b_9):
	return a_0*np.exp(-b_0*(r-10)**2)+a_1*np.exp(-b_1*(r-10)**2)+a_2*np.exp(-b_2*(r-10)**2)+a_3*np.exp(-b_3*(r-10)**2)+a_4*np.exp(-b_4*(r-10)**2)+a_5*np.exp(-b_5*(r-10)**2)+a_6*np.exp(-b_6*(r-10)**2)+a_7*np.exp(-b_7*(r-10)**2)+a_8*np.exp(-b_8*(r-10)**2)+a_9*np.exp(-b_9*(r-10)**2)



Data = pd.read_csv(datapath, delim_whitespace=True, engine="python", header=None)


r = Data[0].astype(float)
r_Bohr = pd.Series([i*1.8897259886 for i in r])
V_ee = Data[2].astype(float)
V_Total = Data[3].astype(float)

In [45]:
popt_total, pcov_total = curve_fit(gaussians2, xdata=r_Bohr, ydata= V_Total, maxfev=50000)

L = [0 for i in range(0,len(popt_total))]

coefficients= np.split(popt_total,2)[0]
exponents= np.split(popt_total,2)[1]

centers1 = [[0.0,0.0,0.0] for i in range(0,len(popt_total))]
centers2 = [[10.0,0.0,0.0] for i in range(0,len(popt_total))]

write_potential_to_lowdin_format("FCENTER","E+",L,exponents,coefficients,centers1)
write_potential_to_lowdin_format("SCENTER","E+",L,exponents,coefficients,centers2)


In [47]:
join_potentials("FCENTER","SCENTER","NTOTPOT","E+")